In [14]:
import xarray as xr
from ocf_blosc2 import Blosc2
from tqdm import tqdm
%load_ext autoreload
from torch.utils.data import DataLoader, IterableDataset
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from dataset import HDF5Dataset 
dataset = HDF5Dataset(['/data/processed_data/processed_train_1.hdf5'], "/data/sat_np/", "/data/weather_np/", True, True, True, True)
data_loader = DataLoader(dataset, batch_size=16, pin_memory=True, num_workers=8, shuffle=False)
print(f"train dataset len: {len(dataset)}")

Opening file /data/processed_data/processed_train_1.hdf5.
Warming up the dataloader!


100%|██████████| 3447/3447 [00:00<00:00, 129734.71it/s]

train dataset len: 3447


In [18]:
for (pv_features, hrv_features, nwp_features, extra, pv_targets) in (pbar := tqdm(data_loader, total=len(data_loader), ascii=True)):
    print(hrv_features.shape, nwp_features.shape)

  0%|          | 1/216 [00:03<11:32,  3.22s/it]

torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])


  1%|          | 2/216 [00:06<12:00,  3.37s/it]

torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])
torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])
torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])


  2%|2         | 5/216 [00:06<03:32,  1.01s/it]

...

torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])


 37%|###6      | 79/216 [00:34<01:01,  2.24it/s]

torch.Size([16, 11, 12, 128, 128]) torch.Size([16, 6, 10, 128, 128])


 37%|###6      | 79/216 [00:34<01:00,  2.28it/s]


KeyboardInterrupt: 

In [ ]:
print(dataset[100])

0 100
/data/sat_np/21-01-01/08:00:00.npy
None
